#Van Gestel's Simulation Model (conversion from OHT to POAG) in Python 
### Single patient simulation
To demonstrate the logic of the project, we would simulate one patient, how he move within the system and also how the different methods and procedures change his attributes

For a more general overview, it would be elsewhere using either UML activity diagram or sequence diagram. This iPython is more notebook for verification of actual codes' logic 

To prevent accidental running of codes (and false-alarm error messages), if needs be, some demonstration snippets would be inside the `if False` statement so that it will not be executed. We would indicate these blocks of codes. The corresponding procedures will be imported and run from the **actual** project files

#Content 
####1. `MainProgram.py` file and overall design of the project
####2. `PatientClass.py`
    2.a. Step 3 inCurredSideEffect
    
    2.b. Step 5
    
    2.c  Step 7
    
####3. `DoctorClass.py`
####4. `MedicationBlockClass.py`
####5. `Trabeculectomy` and `Implant` objects

##1.  `MainProgram.py` file and overall design of the project
The main logic of the program resides in `MainProgram.py` file 

We would take a look at this file first. The codes will not run 

In [1]:
"""
    File: MainProgram.py
    [....]
"""

#line 40 onwards: 
if False: 
        plottingsystem = PlottingSystem(plt)
        #order variable is to keep track of the number of plot grids that
        #our system is drawing on
        order = 1
        for i in range(numberPatientPopulation):
            sysSimulation = SimulationSystem(3000,"PatientList/Patients_list_{}.csv".format(i))
            sysSimulation.SystemSimulation()
            plottingsystem.plot(sysSimulation,order,i,masterListforReplications)
            order += (numberofGraphs*1) 
            del sysSimulation 
        print order 
        csv_dict_writer("MList.csv",field_names,masterListforReplications)

Each run in the `MainProgram.py` will simulate `numberPatientPopulation` of patients' populations
Correspondingly, there will be `numberPatientPopulation` of simulation objects - which are called `sysSimulation` objects here

The initial parameters recorded in the **Patients_list_x.csv** file, and is referenced to `sysSimulation` object for actual simulation 

`plottingsystem` object is created only once for the entire run. And it will handle the plotting of all the `numberPatientPopulation` of runs

**Main logic** of the project resides in the `sysSimulation` object and its dependencies, let's take a look at the interface of  `SimulationSystem` object, which is `SystemSimulation` method. 

In [2]:
"""
    File: SimulationSystemClass.py
    [.....]
"""
"""Code snippet samples are here"""
#line 54
def SystemSimulation (self):
        """
        Read in the initial parameters from file_name
        Initiate correct initial parameters for patients
        """
        #read in the records of patients' initial parameters 
        #specified in the path self.file_name
        with open(self.file_name) as f_obj:
            self.csv_dict_reader(f_obj)
        env = simpy.Environment()   
        
        for i in range(self.size):
            # patients are initialized and added to the patientlist 
            self.patientlist.append( Patient(env,i,self.monitor,{'IOP':self.list_IOP[i],'MD': self.list_MD[i],
            'MDR':self.list_MDR[i],'CumulativeMDR': 0,'IOPTarget': 24,'Age':self.list_Age[i], 
            'TrabeculectomyIOP': 0,'Death':self.list_Death[i],'Gender': self.list_Gender[i]}))
        env.run(until = simulationTime)
        self.final_cost_calculate ()

This method would first call method `.csv_dict_reader(...)` to obtain the initial patients' parameters and store these parameteres as the `list` object attributes of the `SimulationSystem` object  
        
Then the method would initialize the patient objects when their corresponding initial parameters from the class attribute lists: `self.list_IOP, self.list_MD`. 

Patient all receives the env object so that simpy can schedule the simulation. env starts to run using  `env.method`

The actual work is done in the **Patient** object, which is discussed below

##2. `PatientClass.py` file
Each patient is simulated individually of each other

Once declared, patient would automatically schedule to be simulated by running command `.action = env.process(self.runSimulation())`

Overall general flow of patients in the system can be summarized as follows:

    1. Patient visits doctor and have his medical records/treatment changed if there is an incident (side-effect, IOP > Target etc).

    2. Patient changes his personal attributes (IOP, MD etc) using these medical attributes 

    3. Repeat the simulation until patient's death

Most of the codes that execute this flow resides in both `PatientClass.py` and `DoctorClass.py` which contain the   **Patient** and **Doctor** module

A  visualization of how **Patient** and **Doctor** interact here in the activity diagram:

<img src = "Images/ActivityDiagram1.JPG">
*Figure 1. Logic sequence of patient and doctor interactions*

The UML sequence diagram in the documentation would expose the complete API of the project

##Now, let's intialize a single patient and see how his attributes are updated throughout the simulation. 



In [2]:
"""Here are objects are imported from the actual project files"""
from PatientClass import Patient
from MonitorClass import Monitor
import simpy 
monitor = Monitor(30)
env = simpy.Environment()
#Our MDR is monthly decrease
patient = Patient(env,1,monitor,{'IOP':32,'MD': -2.3,
            'MDR':0.02,'CumulativeMDR': 0,'IOPTarget': 24,'Age':67, 
            'TrabeculectomyIOP': 0,'Death':100,'Gender': 1})

"""We also need to call our doctor to treat the patient"""
from DoctorClass import Doctor
doctor = Doctor(patient.Attribute,patient.params,patient.medicalRecords)

Let's familiarize with the attributes of **Patient** object. These attributes would be changed during the simulation of the patient, depending on his current medical status. 

In [ ]:
"""Here are only demonstration code snippets"""

"""
    File PatientClass.py
"""
#line 38 
#1. 
        self.monitor = monitor
        
        """
        The 3 lists of patients: 
                 self.Attribute: personal list containing IOP,MD,Age etc
                 self.params: medical parameters 
                 self.medicalRecords: medical related records
        """
#2. 
        self.Attribute = Attribute
        self.params = {'IOPReduction':0,'time_next_visit': 0,'FirstProgression':0,
                       'SecondProgression':0,'VFCountdown':0,
                       'SideEffect':0,
                       'CurrentTime' : 0, 'Conversion': False,'CurrentMonth':0}
        self.medicalRecords = {'PatientVisits': 0, 'MedicationIntake': 0,
                               'MedicationCombination':[0,0,0,0,0],
                               'CurrentMedicationType':0,'TreatmentOverallStatus': 0,
                               'TreatmentBlock': 'A', 'MedicationPath': [0,0,0,0,0],
                               'ContinueTreatment':True,
                               'NumberVF':0, 
                               'ExitCode': False,
                               'NumberTrabeculectomy':0, 'TrabeculectomySuccess': True,
                               'OnTrabeculectomy': False,
                               'ImplantSuccess': True, 'OnImplant':False,
                               'Cataract' :False,'SurgeryCataract':0,
                               'Diagnosed' : False}
                               
                               
        self.QALY = 0

The `monitor` object in the `#1` part is used to collect patient's statistics throughout the simulation. We would discuss more of it in appendix **A**

The following 4 attributes (in `#2`) are of important concern throughout the simulation. `self.Attribute` is the personal attribute of the patient that contain records of his/her current IOP,MD,Age etc. This attribute is mainly modified by **Patient** object itself. `self.params` and `self.medicalRecords` are mainly modified by **Doctor** object and reflect the changes in treatment plan for the **Patient**

`.QALY` is to record the current patient's Quality-of-Life of **Patient** object

Let's take a look at the interface  of the **Patient** object - which resides in the `runSimulation` method 

In [2]:
"""
    File PatientClass.py
"""
#line 70
def  runSimulation (self):
        """
        While patient is still alive: 
            1. Update Current attributes (either monthly or by visit)
            2. Visit doctor
            3. From the current medication, incur side effect
            4. Calculate current cumulative medical costs incurred
            5. Update patient's personal attributes (IOP,MD) from the current medication
            6. Time out(delay)
            7. Check for conversion and cataract
        """
        while (self.Attribute['Age'] < self.Attribute['Death']):
            self.VisitUpdate()
            for i in range(int(float(self.params['time_next_visit']))):
                self.MonthlyUpdate()
        #2. 
            doctor = Doctor(self.Attribute,self.params,self.medicalRecords)
            doctor.ReturnAllDoctorValues()
        #3.
            self.inCurredSideEffect(doctor)
        #4.........
        #................................................

From the comments of the codes, it implies that steps **2,3,5 and 7** are important to the logic of the model. The other steps are mainly to collect the statistics or keeping the logisitics of the projectp[] - and would be discussed elsewhere in appendix **A**

We will go through steps **2 , 3 , 5 , 7** as they are more important to the logic of the project. 

*Step 2. Visit Doctor* would lead us to a different `python` file. So we will go through *step 2* and `DoctorClass.py` later in part 3. 

We are then left with `Step 3, Step 5 and Step 7`

Using *Figure 1* of our project's **Logic sequence**, here is where we are going to focus on:

<img src = "Images/PatientPart.JPG">
*Figure 2. Patient logic part*


##2. a. Step 3: `.inCurredSideEffect` method

**Logic/Method**: 


In [3]:
"""Here is the demonstration code snippet"""
def inCurredSideEffect(self,doctor):
        """Check for Side effect. If yes, immediate visit doctor to change treatment"""
        SideEffect = 0  
        # There will be 'SideEffect' % that patient will incur side effect
        if random.uniform(0,1) < self.params['SideEffect']:
            self.medicalRecords['TreatmentOverallStatus'] = 1
            self.medicalRecords['ContinueTreatment'] = True
            #Just call the doctor, no need to check for IOP target
            doctor.DoctorModule()
            self.monitor.UpdateCurrentMedicationType(self.name,self.medicalRecords)
            self.monitor.UpdateOverallStatus(self.name,self.medicalRecords)
            SideEffect = 1
        """ Update current QALY according to side effect"""
        self.QALY += (0.88 - 0.101*SideEffect + 0.011*self.Attribute['MD'] - 0.065*self.medicalRecords['Cataract'])*(self.DiscountRate()/12)
        
        

**QALY** needs to be calculated within Side effect method because **QALY** depends on chance of the event that side effect happens. 


-----------------------------------------------------------------------------------------------------------------
Let's call the actual codes from our project and test out `inCurredSideEffect`

Since the patient in our system has not visited the doctor (he must have been there in our real simulation system), we will set an arbitrary Side effect % first

In [19]:
#arbitrary Side effect %
patient.params['SideEffect'] = 0.9 
print ("Here is our patient's current Side Effect percentage {}".format(patient.params['SideEffect']))
print ("His current medication type is: {}".format(patient.medicalRecords['CurrentMedicationType']))
patient.inCurredSideEffect(doctor)
if patient.medicalRecords['TreatmentOverallStatus'] == 1:
    print ("This print statement will not be called!")
print ("The reason is because: side effect is {} currently".format(patient.params['SideEffect']))
print ("The patient has his medication changed already due to the Side Effect occurence")
print ("His current medication type is: {}".format(patient.medicalRecords['CurrentMedicationType']))

Here is our patient's current Side Effect percentage 0.9
His current medication type is: 1
The reason is because: side effect is 0.0793896424726 currently
The patient has his medication changed already due to the Side Effect occurence
His current medication type is: 2




##2.b. Step 5: `.params_update` method 

**Logic/Summary**: 
    1. Check for conversion. If yes, start to decrease MD. Also, increment 'Age' accordingly 
    2. Check whether the patient is on medication or not. Also whether the patient is on trabeculectomy so as to decrease/increase IOP accordingly
    3. If patient is on LTP only, decrease the effectiveness (IOP reduction) accordingly 
    
    
The relationship between MD and IOP in van Gestel's model is dictated as follows: 

**If** IOP > 13: $$Decrease = MDR*1.13^{IOP - 15.5}*months$$

**Else**: $$Decrease = 0$$



In [ ]:
"""Here is the demonstration code snippets"""
def params_update(self):
##1.    
        """Update current IOP and MD based on medical records"""
        #Update MD 
        #If there is conversion, then check whether....
        if self.params['Conversion'] == True:
            #If IOP > 13, then start to decrease MD
            if self.Attribute['IOP'] > 13:
                difference = self.Attribute['MDR'] *(1.13**(self.Attribute['IOP'] - 15.5))*(self.params['time_next_visit'])
            else:
                difference = 0
            #Cumulative MD reduction. To see whether any progression occurred 
            self.Attribute['CumulativeMDR'] = self.Attribute['CumulativeMDR'] + difference
            self.Attribute['MD'] = self.Attribute['MD'] - difference
        self.Attribute['Age'] = self.Attribute['Age'] + self.params['time_next_visit']/12
##2.
        #Update IOP based on medication/treatment 
        if  self.medicalRecords['ContinueTreatment'] == False or self.params['IOPReduction'] < 0.001:
            self.onNoMedicationOrTrabeculectomy()
        if self.medicalRecords['ContinueTreatment'] == True :
            self.onMedication()
##3. 
        #If patient is on LTP only, reduce the effective IOP reduction % overtime 
        if self.medicalRecords['CurrentMedicationType'] == 5 and self.params['IOPReduction'] > 0:
            self.params['IOPReduction'] -= self.params['IOPReduction']*(self.params['time_next_visit']/12)

MD is directly dependent on IOP only

Modification of MD is under 2 different scenarios:

* Patient currently on medication
* Patient currently on no medication or just after trabeculectomy/implant is performed. 

The scenario how IOP is updated is determined using attribute `'ContinueTreatment'` under `dictionary` **medicalRecords**

--------------------------------------------------------------------------------
Now let's check on our real codes, by calling the `params_update` method

In [20]:
print ("Patient's current IOP is {}".format(patient.Attribute['IOP']))
print ("Patient's current MD is {}".format(patient.Attribute['MD']))
patient.params_update()
print ("")
print ("Patient's current IOP is {}".format(patient.Attribute['IOP']))
print ("Patient's current MD is {}".format(patient.Attribute['MD']))

print ("Patient's MD should not be decreased now since there is no conversion yet")

Patient's current IOP is 32
Patient's current MD is -2.3

Patient's current IOP is 29.6213577859
Patient's current MD is -2.3
Patient's MD should not be decreased now since there is no conversion yet



##2.c. Step 7: Check for conversion and Cataract

In van Gestel's simulation model, patients are all initiated as OHT. There is a **annual** survival probability that the patient would be converted to POAG. This probability is given as $$P = 1 - S =  1 - e^{h_{i}t}$$ 
where $$h_{i} = 1.26^{\frac{age - 55}{10}}*1.09^{IOP-24} *0.02$$ and $t$ is the current time in year(s) since patient's entering the system

this conversion probability would be run at discrete event during patient's wait before seeing the doctor


In [21]:
"""Demonstration code snippets"""
def EvaluateConversion (self):
        """Evaluate survival probability to conversion from OHT to POAG"""
        h = (1.26**((self.Attribute['Age']-55)/10))*(1.09**(self.Attribute['IOP'] - 24))*0.02
        prob = 1 - math.exp(-h*self.params['CurrentTime'])
        if random.uniform(0,1) < prob:
            self.params['Conversion'] = True

Let's test our function on the actual patient, using the actual `EvaluateConversion` method

In [24]:
"""Actual project method"""
patient.params['CurrentTime'] = 50
patient.EvaluateConversion()
print ("Here is the current conversion status of patient {}".format(patient.params['Conversion']))
print ("Let's call params update again")
patient.params_update()
print ("Patient's current IOP is {}".format(patient.Attribute['IOP']))
print ("Patient's current MD is {}".format(patient.Attribute['MD']))
if patient.params['Conversion'] == False:
    print ("Since conversion is False, patient MD has decreased yey")
else:
    print ("Now MD is decreased")

Here is the current conversion status of patient True
Let's call params update again
Patient's current IOP is 23.4947004617
Patient's current MD is -3.09531556212
Now MD is decreased
50


Cataract occurence and removal surgeries would be run from **Cataract** object interface - which is the `InitiateCataract` method 

`cataract_formation` and `cataract_formation_female` are lists that store the annual incidence rate of patient developing cataract

The code snippets should be self-explanatory using the in-line comments

In [2]:
"""Below is the sample codes from cataract object - showing InitiateCataract interface"""
def InitiateCataract(self):
        key = int((self.Attribute['Age'] -50)/5)
        cataractRisk = random.triangular(1.5,2.7,4.9)
        """This if-else block is to assign the correct incidence cataract rate for the patients"""
        if self.medicalRecords['NumberTrabeculectomy'] == 0:  
            if self.Attribute['Gender'] == 1:
                if key < 7: 
                    RateCataract = (cataract_formation[key]/1000)
                else:
                    RateCataract = (cataract_formation[7]/1000)
            else:
                if key < 7: 
                    RateCataract = (cataract_formation_female[key]/1000)
                else:
                    RateCataract = (cataract_formation_female[7]/1000)
        else:
            if self.Attribute['Gender'] == 1:
                if key < 7: 
                    RateCataract = (cataract_formation[key]/1000)*cataractRisk
                else:
                    RateCataract = (cataract_formation[7]/1000)*cataractRisk
            else:
                if key < 7: 
                    RateCataract = (cataract_formation_female[key]/1000)*cataractRisk
                else:
                    RateCataract = (cataract_formation_female[7]/1000)*cataractRisk
                    
        """evaluate cataract incidence"""            
        if random.uniform(0,1) <RateCataract:
            self.medicalRecords['Cataract'] = True
        """evaluate incidence of cataract surgeries"""
        if random.uniform(0,1) < random.beta(123,109) and self.medicalRecords['Cataract'] == True:
            self.medicalRecords['SurgeryCataract'] += 1
            self.medicalRecords['Cataract'] = False

##3. `DoctorClass.py` file

This is *Step 2* in the `runSimulation` method of **Patient** object


We saw that **Patient** module would call the `.ReturnAllDoctorValues()` methods to update current treatment of the patient.

First let's initialize a doctor module from our **actual** project file

In [8]:
"""
    File: DoctorClass.py 
    
"""
#line 38 
def ReturnAllDoctorValues (self):
        """Main interface of the class
        1. Set IOP target, if the time for VF comes
        2. Evaluate IOP against target and (possibily) nullfify %side effect
        3. Go to meet doctor
        """
        self.IOPTargetSetting()
        self.IOPandSideEffectEvaluation()
        self.DoctorModule()

Let's go through these 3 methods one by one and see their effect on our patient
##3.a. `IOPTargetSetting` method 

Let's first take a look at the demonstration code snippets

In [3]:
"""This is the demonstration snippets of code logic"""
def IOPTargetSetting(self):
        """Check whether its time for VF test. If yes, reevaluate IOP target"""
        if self.params['VFCountdown'] > TimeToVFTest: 
            self.SetCorrectIOPTarget()
            self.medicalRecords['NumberVF'] +=1
        self.params['VFCountdown'] = self.params['VFCountdown'] + self.params['time_next_visit']



In [37]:
"""Here we call the actual doctor to Evaluate IOP Target of our one patient"""
doctor.IOPTargetSetting()
print (" Here is the patient params:    {}".format(patient.params))
print (" Here is the patient Attributes:  {}".format(patient.Attribute))
patient.Attribute['IOP'] = 27
patient.params_update()
print ("")
print ("His IOP Target is currently {} ".format(patient.Attribute['IOPTarget']))
print (patient.params['VFCountdown'])

 Here is the patient params:    {'SecondProgression': 0, 'SideEffect': 0.07938964247261476, 'Conversion': True, 'IOPReduction': 0.29733027675721835, 'CurrentMonth': 0, 'time_next_visit': 3, 'CurrentTime': 50, 'FirstProgression': 1, 'VFCountdown': 3}
 Here is the patient Attributes:  {'MD': -4.362102950120095, 'Death': 100, 'CumulativeMDR': 0, 'IOPTarget': 18.0, 'TrabeculectomyIOP': 0, 'MDR': 0.02, 'Gender': 1, 'Age': 69.5, 'IOP': 24.993020631888776}

His IOP Target is currently 18.0 as there is IOP Target evaluation yet
3


*Some explanation*:

As we can see from the `if` condition, when the time comes for VF measurement, the patient's IOP target will be reevaluated using `.SetCorrectIOPTarget` method

`.SetCorrectIOPTarget` method would:

1. evaluate whether there is any progression (cumulative MD decrement of  `2dB`) from either: 
    * the start of the simulation 
    * the lastest progression 
2. reassign IOP Target according to set-up ( treatment alternatives)

We will see how this is done below using the demonstration code snippets

In [26]:
"""Here is the demonstration code snippets"""
def SetCorrectIOPTarget(self):
#1    
        """Evaluate for MD progression. Reset IOP target accordingly"""
        #IOP target is reset stepwise if CumulativeMDR is > 2dB
        if self.params['FirstProgression'] == 1 and self.PatientAttribute['CumulativeMDR'] > 2: 
            self.params['SecondProgression'] =1 
            #Need to reset CumulativeMDR to reflect van Gestel's definition
            self.PatientAttribute['CumulativeMDR'] = 0
        elif self.PatientAttribute['CumulativeMDR'] > 2:
            self.params['FirstProgression'] = 1
            self.PatientAttribute['CumulativeMDR'] = 0
            
#2            
        #if no conversion: IOP target = 24
        #conversion occurs: IOP target is set according to progression param
        if self.params['Conversion'] == True:
            if self.params['FirstProgression'] == 1 and self.params['SecondProgression']  == 1:
                self.PatientAttribute['IOPTarget'] = ThirdProgressionTarget
            elif self.params['FirstProgression'] == 1:
                self.PatientAttribute['IOPTarget'] = SecondProgressionTarget
            else:
                self.PatientAttribute['IOPTarget'] = FirstProgressionTarget
        #Reset VF countdown. Wait till new VF measurement
        self.params['VFCountdown'] = 0


{'MD': -36.449385879379804, 'Death': 100, 'CumulativeMDR': 0, 'IOPTarget': 18.0, 'TrabeculectomyIOP': 0, 'MDR': 3, 'Gender': 1, 'Age': 68.5, 'IOP': 20.791459962771267}


After `IOPTargetSetting` finished reevaluted our `IOP Target`, the patient's IOP is evaluated against the `IOP Target` by calling `IOPandSideEffectEvaluation`. Let's take a look at the code snippets.  

##3.b. `IOPandSideEffectEvaluation` method

In [ ]:
"""Here are the code snippets demonstration"""
def IOPandSideEffectEvaluation(self):
        """ Compare IOP against target. Change patient overall status accordingly"""
        if self.medicalRecords['MedicationIntake'] > TimenotSideEffect :
            self.params['SideEffect'] = 0
        #If patient's IOP > IOP target, set overall status accordingly 
        #and continue treatment
        if self.PatientAttribute['IOP'] > self.PatientAttribute['IOPTarget']:
            self.medicalRecords['TreatmentOverallStatus'] = 2
            self.medicalRecords['ContinueTreatment'] = True
        else:
        #discontinue treatment
            self.medicalRecords['ContinueTreatment'] =False

Let's see how IOP is re-evaluated against the IOP Target:


In [40]:
"""Here is the actual code run"""
doctor.IOPandSideEffectEvaluation()
print ("Current patient status is {}, patient is successfully reevaluated".format(patient.medicalRecords['TreatmentOverallStatus']))

Current patient status is 2, patient is successfully reevaluated


**Important**: `TreatmentOverallStatus` attribute under **medicalRecords** dictionary is use to direct treatment decisions in the treatment modules - which would be discussed later

##3.c `DoctorModule` method

**Logic**
1. Initialize treatment block and pass patient's parameters for changing. An exit code would be returned. 
2. Evaluate the exit code. If true, change treatment and re-initialize treatment block again. 

This is demonstrated using the bwl code snippets

In [ ]:
def DoctorModule(self):
#1
        """Initialize and/or change treatment block"""
        self.InitializeCorrectTreatment()
#2
        #ExitCode will be returned from one of treatment block.
        #ExitCode == True means either  
        #no more medication left (if in MedicationBlock)
        #or IOP > IOP target in surgeries Block (Trabeculectomy/Implant)
        if self.medicalRecords['ExitCode'] == True:
            self.ChangeTreatmentPlan()
            self.InitializeCorrectTreatment()
            self.medicalRecords['ExitCode'] = False            
        
        self.medicalRecords['PatientVisits'] += 1   

`InitializeCorrectTreatment` will be  attribute `TreatmentBlock` of dictionary `medicalRecords`



In [ ]:
"""Here is the extraction of  InitializeCorrectTreatment"""
def InitializeCorrectTreatment(self):
        """
        Initialize correct treatment block according to 
        letter block of self.medicalRecords['TreatmentBlock']
            A,C,E and G: medication blocks
            B and D: trabeculectomy blocks
            F : Implant block
        order: refer to appendix
        """
        if self.medicalRecords['TreatmentBlock'] == 'A': 
            block = MedicationBlock(self.params,self.medicalRecords)
            block.update()
            del block
        elif self.medicalRecords['TreatmentBlock'] == 'B':
            block = TrabeculectomyBlock(self.PatientAttribute,self.params,self.medicalRecords)
            block.update()
            del block
        elif self.medicalRecords['TreatmentBlock'] == 'C':
            block = MedicationBlock(self.params,self.medicalRecords)
            block.update()
            del block

`ChangeTreatmentPlan` will change the block using a look-up table - implemented as Python dictionary called **GraphPlan**

Bwl is the treament plan

<img src = "Images/Flow.JPG">


In [ ]:
"""This is the graph plan for the next medication correspoinding to current treatment block"""
GraphPlan = {'A':['B','E'],'B': ['C'],'C':['D','E'],'D':['E'],
             'E':['F','G'],'F':['G'],'G':'Terminal'}


"""..................."""
def ChangeTreatmentPlan(self):
        """Change treatment block to the next treatment block once ExitCode is triggered"""
        """
        Logic: 
            if patient is currently on Surgeries, next treatment is medication
            if patient is currently on C(medication), next treatment is either 
                2nd trabeculectomy(block D) if 1st trabeluctomy succeeded
                or medication(block E) if 1st trabeculectomy failed
            if patient is on medication, surgeries (trabeculectomy or implant)
                only occur if self.PatientAttribute['Age'] < AgeNottoSurgery
                else, go to a different medication block
        """ 
        key = self.medicalRecords['TreatmentBlock']
        if key == 'B' or key == 'D' or key == 'F':
            self.medicalRecords['TreatmentBlock'] = GraphPlan[key][0]
        elif key == 'C':
            if self.medicalRecords['TrabeculectomySuccess'] == True:
                self.medicalRecords['TreatmentBlock'] = GraphPlan[key][0]
            else:
                self.medicalRecords['TreatmentBlock'] = GraphPlan[key][1]
        else:
            if self.PatientAttribute['Age'] < AgeNottoSurgery: 
                self.medicalRecords['TreatmentBlock'] = GraphPlan[key][0]
            else:
                self.medicalRecords['TreatmentBlock'] = GraphPlan[key][1]

The **idea** of this  is first refer to the current treatment block, using **key**. Then, depending on his status, direct the next treatment type.

##4. Medication Block 

**Assumption**: Medication only changed when `TreatmentOverallStatus` != 0 (which means there is a special event occuring to patients). Then, method `DoctorPrescription` will be called 

In van Gestel's paper, patients are indicated medications using a plan as below: 
<img src = "Images/Capture.JPG">

The plan is implemented as **binary tree** as at any node, there are two options for patients to take. 

The tree is a collection of nodes. The node class consists of 3 attributes: `.value`, `.left`, `.right` 

`.value` will be a list in the form [list,int,list,list] = [medication effectiveness, medication type, medication side effect, medication combination indicator]

**Important**: value of each node would be extract and use to set correct parameters for the patient using method **operations**

`.left` refers to the **node** object on the left of this current node

`.right` refers to the **node** object on the right of that node. 

The whole medication plan is hard-coded into the **TreatmentTree** please refer to that. 

Another attribute of interest is `MedicationPath` of dictionary **medicalRecords**

             
Parameters setting will be done in the method `operations`

In [ ]:
def operations(self,tracenode):
        """
        Assign IOP reduction,Current med type, Side Effect and Med combi using
            the values from tree node. 
        """
        self.params['IOPReduction'] = random.beta(tracenode.value[0][0],tracenode.value[0][1])
        self.medicalRecords['CurrentMedicationType'] = tracenode.value[1] 
        if self.medicalRecords['CurrentMedicationType'] <> 5:
            self.params['SideEffect'] = random.beta(tracenode.value[2][0],tracenode.value[2][1])
        else:
            self.params['SideEffect'] = 0
        self.medicalRecords['MedicationCombination'] = tracenode.value[3]

###How to trace medication node using `DoctorPrescription` method


**Assumptions:** Medication must be changed once patient enters this block
#####Basic logic of DoctorPrescription. 
    If no treatment: 
        Initialize treatment, set MedicationPath[0] = 1
    Else: 
        Start tracing to the medication node that patient is currently on
        If patient is currently not on terminal node:
            Evaluate `TreatmentOverallStatus` of dictionary medicalRecords
            If TreatmentOverallStatus == SideEffect
                Trace to the left node 
                Set parameters
                Set the corresponding MedicationPath index
            Else
                Trace to the right node
                Set parameters
                Set the corresponding MedicationPath index
         Else: 
             Exit Code = True, leave medication

Now let's look at how some patient with no medication (just start to enter `Medication` block) have his medication changed

In [5]:
    if False:
        """Reassign/assign medication to patient when unusual event/no medication yet """
        tracenode  = node(0)
        tracenode = TreatmentTree
        # .medicalRecords['MedicationPath'][0] == 0 means no medication assigned yet
        if self.medicalRecords['MedicationPath'][0] == 0:
            #assign first medication and block
            self.medicalRecords['MedicationPath'][0] = 1
            self.operations(TreatmentTree)
            self.medicalRecords['TreatmentOverallStatus'] = 0
            self.medicalRecords['MedicationIntake'] = 0

Below is how someone with some medication history has his Medication changed: 

**First**, check whether patient is on `Medication` type 10, if Yes, Exit immediately

In [ ]:
#special case (premature exit), if current medication type (before reassigning) 
#is 10, Exit 
#because there is no more medication node to take.
if  self.medicalRecords['CurrentMedicationType'] == 10 and self.params['FirstProgression'] == 1:
    self.medicalRecords['ExitCode'] = True

**After this**, we trace to the current node of medication that patient is on:

In [ ]:
#now, trace to the current medication block
#while not exhaust all medication nodes on the path or not medication type == 10
            while i < 5 and self.medicalRecords['MedicationPath'][i] <> 0 and self.medicalRecords['CurrentMedicationType'] <> 10:  
                if self.medicalRecords['MedicationPath'][i] == 1:
                    tracenode = tracenode.left
                    i = i +1
                elif self.medicalRecords['MedicationPath'][i] == 2:
                    tracenode = tracenode.right
                    i = i+1

If this is not the last node on medication block, assign the corresponding medication to patient's `TreatmentOverallStatus`: 

In [ ]:
#if current medication type (before reassigning) is not the terminal node
#first, trace to the next medication node depending on overall status
#then reassign .operations(tracenode)
            if i < 5 and self.medicalRecords['CurrentMedicationType'] <> 10 :
                self.medicalRecords['MedicationPath'][i] = self.medicalRecords['TreatmentOverallStatus']
                if self.medicalRecords['TreatmentOverallStatus'] == 1:
                    tracenode = tracenode.left
                    self.operations(tracenode)
                if self.medicalRecords['TreatmentOverallStatus'] == 2:
                    tracenode = tracenode.right
                    self.operations(tracenode)

If patient is currently on the last node of his path (befor having his medication changed), **Exit**

In [ ]:
#if current medication type (before reassigning) is the terminal node
if i == 5 and self.params['FirstProgression'] == 1:
    self.medicalRecords['ExitCode'] = True
    self.ResetMedicationPath()

Finally, before patient leaves the `DoctorPrescription` method, reset the two attributes below to indicate that patient is on a new, fresh type of medication

In [ ]:
#Reset status variables
self.medicalRecords['TreatmentOverallStatus'] = 0
self.medicalRecords['MedicationIntake'] = 0

Here is how the actual codes will work 

In [9]:
"""Here is the actual project codes"""
from MedicationBlockClass import MedicationBlock
print ("Current medication type is {}".format(patient.medicalRecords['CurrentMedicationType']))
block = MedicationBlock(patient.params,patient.medicalRecords)
block.DoctorPrescription()
print ("Current medication type is {}".format(patient.medicalRecords['CurrentMedicationType']))


Current medication type is 1
Current medication type is 1


The above block of codes will not change `CurrentMedicationType` of the patient as there is no special event that change the `TreatmentOverallStatus` attribute. 

Here is how `DoctorPrescription` would change the medication type.

In [15]:
patient.medicalRecords['TreatmentOverallStatus'] = 1



Now then we would see the change in medication type


In [19]:
block.DoctorPrescription()
print ("Current medication type is {}".format(patient.medicalRecords['CurrentMedicationType']))

Current medication type is 5


###Set number of visits

Number of visits would depend on `CurrentMedicationIntake` of dictionary `medicalRecords`

In [ ]:
def SetTimeNextVisit(self):
    """Set next time to visit doctor for patients on medication block"""
    if self.medicalRecords['MedicationIntake'] == 0:
        self.params['time_next_visit'] = 3
    else:
        self.params['time_next_visit'] = 6

##5. `Trabeculectomy` and `Implant` objects

These two objects are almost the same and quite simple.

The interface of the objects are `.update` method 

**Logic**

    If have not been operated yet
        .SurgeryTE (operating TE)
    Else: 
        Set number of months till next visit
        
    Check whether IOP > IOP target. If yes, leave the module

In [ ]:
    def update(self):
        """
        Interface: 
        After trabeculectomy is done, nothing is done besides set number of visit
        until IOP > IOP target
        """
        if self.medicalRecords['OnTrabeculectomy'] == False:
            self.SurgeryTE()
        else:
            self.SetNumberofVisits()
        #Check whether IOP > IOP target yet. Then evaluate whether to leave the block    
        self.DeterminetoExitTEblock()

Here is how Trabeculectomy is operated. Patient's `IOP` would be reset using a `normal` distribution. Other parameters are the various "house-keeping" variables

In [ ]:
def SurgeryTE(self):
    #reassign the IOP based on the normal distribution reported by van Gestel
    self.Attribute['IOP'] = random.normal(12.5,0.3)
    self.medicalRecords['ContinueTreatment'] = False
    self.medicalRecords['OnTrabeculectomy'] = True
    self.medicalRecords['MedicationIntake'] = 0
    self.medicalRecords['NumberTrabeculectomy'] +=1
    self.medicalRecords['CurrentMedicationType'] = 30
    self.medicalRecords['TreatmentOverallStatus'] = 0
    #nullify any side effect left as patients no longer on medication
    self.params['SideEffect'] = 0

**Implant** object is inherited from Trabeculectomy (to share the `.SetNumberofVisits` method) and share the same structure 

In [14]:
from DoctorClass import Doctor
doctor = Doctor(patient.Attribute,patient.params,patient.medicalRecords)
doctor.ReturnAllDoctorValues()
patient.params_update()
print (patient.Attribute)
print (patient.params)
print (patient.medicalRecords)

{'MD': -2.3, 'Death': 100, 'CumulativeMDR': 0, 'IOPTarget': 24, 'TrabeculectomyIOP': 0, 'MDR': 3, 'Gender': 1, 'Age': 67.75, 'IOP': 22.356061625644106}
{'SecondProgression': 0, 'SideEffect': 0.12151103754432913, 'Conversion': False, 'IOPReduction': 0.28201431593993515, 'CurrentMonth': 0, 'time_next_visit': 6, 'CurrentTime': 0, 'FirstProgression': 0, 'VFCountdown': 12}
{'TreatmentBlock': 'A', 'Diagnosed': False, 'CurrentMedicationType': 1, 'OnTrabeculectomy': False, 'SurgeryCataract': 0, 'MedicationIntake': 2, 'NumberTrabeculectomy': 0, 'MedicationCombination': [1, 0, 0, 0, 0], 'NumberVF': 1, 'ImplantSuccess': True, 'TreatmentOverallStatus': 2, 'TrabeculectomySuccess': True, 'PatientVisits': 3, 'MedicationPath': [1, 0, 0, 0, 0], 'Cataract': False, 'ContinueTreatment': True, 'OnImplant': False, 'ExitCode': False}
